In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
from test_model import test
from argparse import Namespace
import glob
import numpy as np
from concurrent import futures
import multiprocessing
import copy
import json

In [3]:
args = {
    'data_path': '../../Data/',
    'pretrained_model': 'supcon', #'resnet',
    'pretrained_path': '',
    'window_size': 29,
    'strided': 15,
    'batch_size': 512,
    'num_workers': 8,
    'trial_id': 1,
    'ckpt' : 200, #50
    'car_model': None
}

args = Namespace(**args)

In [20]:
bs = 512
pretrained_paths = sorted(glob.glob(f'../save/SupCon.resnet18*_bs{bs}*'))
# pretrained_paths = sorted(glob.glob('../save/smallresnet18.ce?_*_lr0.001_*'))

pretrained_paths

['../save/SupCon.resnet18.ce1_lr0.05_0.01_bs512_200epoch_temp0.07_052122_214019_cosine_warm',
 '../save/SupCon.resnet18.ce2_lr0.05_0.01_bs512_200epoch_temp0.07_052322_102305_cosine_warm',
 '../save/SupCon.resnet18.ce3_lr0.05_0.01_bs512_200epoch_temp0.07_052322_102250_cosine_warm',
 '../save/SupCon.resnet18.ce4_lr0.05_0.01_bs512_200epoch_temp0.07_052322_102242_cosine_warm',
 '../save/SupCon.resnet18.ce5_lr0.05_0.01_bs512_200epoch_temp0.07_052322_102334_cosine_warm']

In [21]:
num_cpu = multiprocessing.cpu_count()
workers = max(num_cpu, len(pretrained_paths))

with futures.ProcessPoolExecutor(workers) as executor:
    to_do = []
    for path in pretrained_paths:
        args.trial_id = path.split('_')[0][-1]
        args.pretrained_path = f'{path}/models/'
        future = executor.submit(test, copy.deepcopy(args), 
                                 {'verbose': False, 'is_cuda': True})
        to_do.append(future)
        
    
    total_results = {}
    for future in futures.as_completed(to_do):
        try:
            results = future.result()
            for k, v in results.items():
                total_results.setdefault(k, [])
                total_results[k].append(v)
        except Exception as error:
            print('An exception occurred: {}'.format(error))

Loading:  SupConResNet
Loading:  SupConResNet
Loading:  SupConResNet
Loading:  SupConResNet
Loading:  SupConResNet
Loading:  LinearClassifier
Loading:  LinearClassifier
Loading:  LinearClassifier
Loading:  LinearClassifier
Loading:  LinearClassifier


  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

  0%|          | 0/648 [00:00<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

  0%|          | 0/648 [00:01<?, ?it/s]

['0.0015', '0.0092', '0.0310', '0.0687', '0.0628']
['1.0000', '0.9999', '0.9997', '0.9993', '0.9994']
['0.9997', '1.0000', '1.0000', '0.9999', '1.0000']
['0.9998', '1.0000', '0.9998', '0.9996', '0.9997']
['0.0010', '0.0046', '0.0504', '0.0528', '0.0483']
['1.0000', '1.0000', '0.9995', '0.9995', '0.9995']
['0.9997', '1.0000', '1.0000', '1.0000', '1.0000']
['0.9999', '1.0000', '0.9997', '0.9997', '0.9997']
['0.0000', '0.0046', '0.0504', '0.0581', '0.0531']
['1.0000', '1.0000', '0.9995', '0.9994', '0.9995']
['0.9997', '1.0000', '1.0000', '1.0000', '1.0000']
['0.9999', '1.0000', '0.9997', '0.9997', '0.9997']
['0.0029', '0.0138', '0.0349', '0.0449', '0.0628']
['1.0000', '0.9999', '0.9997', '0.9996', '0.9994']
['0.9997', '1.0000', '1.0000', '0.9998', '1.0000']
['0.9999', '0.9999', '0.9998', '0.9997', '0.9997']
['0.0010', '0.0184', '0.0310', '0.0502', '0.0580']
['1.0000', '0.9998', '0.9997', '0.9995', '0.9994']
['0.9997', '1.0000', '1.0000', '1.0000', '1.0000']
['0.9999', '0.9999', '0.9998', 

In [22]:
print('Final results')
total_results = {k: np.stack(v, axis=0) for k, v in total_results.items()}
for k, v in total_results.items():
    print(k, list("{0:0.4f}".format(i) for i in v.mean(axis=0)))

Final results
fnr ['0.0013', '0.0101', '0.0395', '0.0549', '0.0570']
rec ['1.0000', '0.9999', '0.9996', '0.9995', '0.9994']
pre ['0.9997', '1.0000', '1.0000', '0.9999', '1.0000']
f1 ['0.9999', '0.9999', '0.9998', '0.9997', '0.9997']


In [18]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [19]:
file = f'small_{args.pretrained_model}{bs}.json'
with open(f'../reports/{file}', 'w') as convert_file:
     convert_file.write(json.dumps(total_results, cls=NumpyEncoder))